In [10]:
import sys
# Before using tensorflow:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import tensorflow as tf
import tensorflow.keras.backend as KB
from matplotlib import pylab as plt
import tensorflow.keras
#from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()

import pickle
import numpy as np
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")

gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.3.2
Keras Version: 2.4.0

Python 3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 16:22:27) 
[GCC 9.3.0]
GPU is NOT AVAILABLE


## Create the model / CNN architecture

In [1]:
import tensorflow
from client.models.AMLmodel import construct_model
model = construct_model()

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 32)      1184      
_________________________________________________________________
activation (Activation)      (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 98, 98, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 49, 49, 64)        1

## Use training data from one clinic to train a model 

In [4]:
import numpy as np
from client.data.datagenerator import DataGenerator

labels0 = np.load('dataset/processed/data_partitions/partition0/labels.npy', allow_pickle=True).item()
data_path0 = 'dataset/processed/data_partitions/partition0/data_singlets'

ids0 = [l for l in labels0]
np.random.shuffle(ids0)
train_split_index = int(len(ids0)*0.9)
train_ids = ids0[:train_split_index]
val_ids = ids0[train_split_index:]

train_gen = DataGenerator(train_ids,labels0, data_path0, dim=(100,100), batch_size=32)
val_gen = DataGenerator(val_ids,labels0, data_path0, dim=(100,100), batch_size=32)

In [5]:
model.fit(train_gen,epochs=20)

Epoch 1/20
17/17 [==============================] - 12s 715ms/step - loss: 1.9728 - accuracy: 0.4393
Epoch 2/20
17/17 [==============================] - 12s 707ms/step - loss: 1.2845 - accuracy: 0.6746
Epoch 3/20
17/17 [==============================] - 12s 697ms/step - loss: 0.9901 - accuracy: 0.7096
Epoch 4/20
17/17 [==============================] - 12s 697ms/step - loss: 0.8417 - accuracy: 0.7224
Epoch 5/20
17/17 [==============================] - 12s 697ms/step - loss: 0.5939 - accuracy: 0.8254
Epoch 6/20
17/17 [==============================] - 12s 701ms/step - loss: 0.4592 - accuracy: 0.8401
Epoch 7/20
17/17 [==============================] - 12s 702ms/step - loss: 0.3362 - accuracy: 0.8695
Epoch 8/20
17/17 [==============================] - 12s 692ms/step - loss: 0.2665 - accuracy: 0.9099
Epoch 9/20
17/17 [==============================] - 12s 685ms/step - loss: 0.2073 - accuracy: 0.9393
Epoch 10/20
17/17 [==============================] - 12s 686ms/step - loss: 0.1534 - accura

## Model validation - plot a confusion matrix

We reserve partition1 as a validation set 

In [15]:
np.sort([k for k in labels0])

labels1 = np.load('dataset/processed/data_partitions/partition1/labels.npy', allow_pickle=True).item()
data_path1 = 'dataset/processed/data_partitions/partition1/data_singlets'

ids1 = [l for l in labels1]
np.random.shuffle(ids1)

test_gen = DataGenerator(ids1,labels1, data_path1, dim=(100,100), batch_size=32)
labels, preds = test_gen.predict_all(model)

In [16]:
def confusion_matrix(model, data_gen):
    
    labels, preds = data_gen.predict_all(model)

    y_pred = np.argmax(preds,1)
    y_data = np.argmax(labels,1)
    M = np.zeros((16,16))

    for pred_, true_ in zip(y_pred,y_data):
        M[true_,pred_] +=1
    
    M[15,:15] = np.sum(M[:15,:15],0)
    M[:15,15] = np.sum(M[:15,:15],1)
    M[15,15] = np.sum(M[:15,:15])
        
    return M

In [ ]:
import matplotlib
from client.models.AMLmodel import classes as aml_classes

class_names_y = [aml_classes[k] for k in aml_classes] + ['Total']
class_names_x = [k for k in aml_classes] + ['Total']

def plot_confusion_matrix(matrix, title='Confusion matrix', cmap=plt.cm.autumn):
    
    f, ax = plt.subplots(1,1,figsize=(20,14))
    color_mat = np.zeros(matrix.shape)
    color_mat[:15,:15] = matrix[:15,:15]/np.maximum(1,np.sum(matrix[:15,:15],0))
    ax.matshow(color_mat, cmap=cmap, norm=matplotlib.colors.LogNorm()) 

    tick_mark = np.arange(16)
    ax.set_xticks(tick_mark)
    ax.set_yticks(tick_mark)
    ax.set_xticklabels(class_names_x)
    ax.set_yticklabels(class_names_y)
    
    for (i, j), z in np.ndenumerate(matrix):
        ax.text(j, i, '{:0.0f}'.format(z), ha='center', va='center')

    ax.axvline(x=14.5,color='black',lw=5)
    ax.axhline(y=14.5,color='black', lw=5)
    ax.set_xlabel('Predicted class', fontsize=20)
    ax.set_ylabel('True class', fontsize=20)

M = confusion_matrix(model, test_gen)
plot_confusion_matrix(M)
plt.savefig('confusion_sample.png')

## Create a model object in STACKn 

In [21]:
from stackn import stackn
tf.saved_model.save(model, 'models/clinic1/')
stackn.create_object('aml-clinic1', release_type="major")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


10/12/2021 07:35:15 PM [deprecation.py:317] From /opt/conda/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


10/12/2021 07:35:15 PM [deprecation.py:317] From /opt/conda/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py:111: Layer.updates (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: models/clinic1/assets


10/12/2021 07:35:18 PM [builder_impl.py:774] Assets written to: models/clinic1/assets


Failed to create model.
Returned status code: 403
Reason: Forbidden
{"detail":"Authentication credentials were not provided."}


NameError: name 'repo' is not defined

## Save weights for use as an initial model in FEDn 

In [25]:
from fedn.utils.kerashelper import KerasHelper
helper = KerasHelper()
weights = model.get_weights()
helper.save_model(weights, "model_single_clinic.npz")

'model_single_clinic.npz'